# Final Modeling For Case Study 2
Here is the order of things to do:

## Main Model for Differentiating Stress / Amusement -- all vars
- Import final.csv and process into X and Y matrices
- GridSearchCV to find best LogisticRegression with L1 penalty
- Make table of coefficients, p-values, 95% confidence intervals based on this regression
- GridSearchCV to find best XGBoost model 
- Feature importance table from XGBoost
- Try number of SVMs, GridSearchCV on polynomial order
- Plot out CV predicted probas to see if it seems like an ensemble method would work
- Fit PLS, fit classifiers on PLS inputs alone
- Train StackingClassifier ensemble model

- Figure out what tables we want, get the data formatted into CSVs and send them out.

## Chest Only Model
- Same approach as in the first part, only removing the features that aren't in the chest

## Quantify Heterogeneity Across Individuals in Response to Stress vs. Amusement
- Somehow make interaction terms?

In [1]:
import pandas as pd
import numpy as np
np.random.seed(440)
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.cross_decomposition import PLSRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC
from tqdm import tqdm
from collections import defaultdict
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_predict

In [2]:
final = pd.read_csv("final_updated_large.csv")

In [3]:
# Removing columns with NA values
na_cols = []
for col in final.columns:
    numnas = sum(final[col].isna())
    print(col, numnas)
    if numnas > 5:
        na_cols.append(col)
na_cols.extend(["Unnamed: 0"])
final = final.drop(columns = na_cols)
print("Dropped", len(na_cols), "columns")

Unnamed: 0 0
ECG_Rate_Mean 0
HRV_RMSSD 0
HRV_MeanNN 0
HRV_SDNN 0
HRV_SDSD 0
HRV_CVNN 0
HRV_CVSD 0
HRV_MedianNN 0
HRV_MadNN 0
HRV_MCVNN 0
HRV_IQRNN 0
HRV_pNN50 0
HRV_pNN20 0
HRV_TINN 0
HRV_HTI 0
HRV_ULF 274
HRV_VLF 274
HRV_LF 274
HRV_HF 3
HRV_VHF 3
HRV_LFHF 274
HRV_LFn 274
HRV_HFn 3
HRV_LnHF 3
HRV_SD1 0
HRV_SD2 0
HRV_SD1SD2 0
HRV_S 0
HRV_CSI 0
HRV_CVI 0
HRV_CSI_Modified 0
HRV_PIP 0
HRV_IALS 0
HRV_PSS 0
HRV_PAS 0
HRV_GI 0
HRV_SI 0
HRV_AI 0
HRV_PI 0
HRV_C1d 0
HRV_C1a 0
HRV_SD1d 0
HRV_SD1a 0
HRV_C2d 0
HRV_C2a 0
HRV_SD2d 0
HRV_SD2a 0
HRV_Cd 0
HRV_Ca 0
HRV_SDNNd 0
HRV_SDNNa 0
HRV_ApEn 0
HRV_SampEn 0
eda_mean_chest 0
eda_std_chest 0
eda_min_chest 0
eda_max_chest 0
eda_slope_chest 0
eda_range_chest 0
eda_mean_scl_chest 0
eda_std_scl_chest 0
eda_std_scr_chest 0
eda_scl_corr_chest 0
eda_num_scr_seg_chest 0
eda_sum_startle_mag_chest 0
eda_sum_response_time_chest 0
eda_sum_response_areas_chest 0
eda_mean_wr 0
eda_std_wr 0
eda_min_wr 0
eda_max_wr 0
eda_slope_wr 0
eda_range_wr 0
eda_mean_scl_wr 0
ed

In [4]:
# Create dummies for subject
final["subject"] = final["subject"].astype("category")
final = pd.get_dummies(final, drop_first = True)

In [5]:
# Convert label to 1 for stress, 0 for amusement
final["label"] = (final["label"] == 2.0).astype(int)
final["label"].value_counts()

1    184
0     90
Name: label, dtype: int64

In [6]:
def inf_to_mean(X):
    """
    Takes numpy array X and returns a version replacing inf and na values with their column means
    """
    X = np.nan_to_num(X, nan = np.nan, posinf = np.nan)
    col_mean = np.nanmean(X, axis = 0)
    inds = np.where(np.isnan(X)) 
    X[inds] = np.take(col_mean, inds[1]) 
    return X

In [7]:
# Format data into numpy arrays to be used in sklearn models
Y = np.array(final["label"])
X = final.drop(columns = ["label"])
X = inf_to_mean(X.to_numpy())

In [8]:
X_colnames = final.drop(columns = ["label"]).columns

In [9]:
best_models = {}

# Lasso Logistic

In [10]:
# Initialize penalized logistic regression
lr = LogisticRegression(max_iter = 10000, penalty = "l1", solver = "liblinear")

In [11]:
# Initialize Cross Validation over L1 penalty coefficient
cv = GridSearchCV(lr, param_grid = {
    "C":[0.01, 0.1, 1, 100, 1000]},
                 scoring = "accuracy",
                 cv = 15
                 )

In [ ]:
# Perform CV
cv.fit(X,Y)

In [ ]:
# Show CV Results
lr_cv_results = pd.DataFrame(cv.cv_results_).sort_values("rank_test_score")
lr_cv_results

In [ ]:
best_lr = cv.best_estimator_
best_models["logistic"] = best_lr

In [ ]:
n_chosen_coefs = len([x for x in best_lr.coef_.reshape(-1) if x != 0])

In [ ]:
# Variables chosen by the Lasso model and their coefficients
print("Coefficients:")
lr_coef_best_idx = np.flip(np.argsort(np.abs(best_lr.coef_).reshape(-1)))
lr_coef_best = best_lr.coef_.reshape(-1)[lr_coef_best_idx]
coef_touse = []
coef_toexclude = []
for idx, coef in zip(lr_coef_best_idx, lr_coef_best):
    if coef != 0:
        print(f"{X_colnames[idx]}: {coef}")
        coef_touse.append(X_colnames[idx])
    else:
        coef_toexclude.append(X_colnames[idx])

# Find Best XGB Model

In [ ]:
# Initialize XGBoost Model
xgb = XGBClassifier()

In [ ]:
# Set parameter grid for cross-validation
xgb_params = {
    "eta":[0.01,0.1,0.2],
    #"min_child_weight":[1, 5, 10],
    "max_depth":list(np.arange(3,11, 2)),
    "gamma" : [0, 0.1, 0.5],
    "subsample":[0.5,1],
    "colsample_bytree":[0.5,1],
    "alpha":[0,1,10,100]
}

In [ ]:
# Initialize cross-validation over selected parameters
cv = GridSearchCV(xgb, param_grid = xgb_params,
                 scoring = "accuracy",
                 cv = 15
                 )

In [ ]:
cv.fit(X,Y)

In [ ]:
xgb_cv_results = pd.DataFrame(cv.cv_results_).sort_values("rank_test_score")
xgb_cv_results

In [ ]:
best_xgb = cv.best_estimator_
best_models["xgb"] = best_xgb

In [ ]:
fi = best_xgb.feature_importances_
fi_best_idx = np.flip(fi.argsort())
fi_best = np.flip(np.sort(fi))
print("MOST IMPORTANT FEATURES:\n")
for i in range(len(fi_best_idx)):
    print("Feature name: {:>12}     Feature importance: {:>12}".format(X_colnames[fi_best_idx[i]], fi_best[i]))

# Calculation of SVM Models

In [ ]:
cv_linear_svm = GridSearchCV(SVC(kernel = "linear"), param_grid = {
    "C":[1, 10, 100]
})
cv_linear_svm.fit(X,Y)
best_linear_svm = cv_linear_svm.best_estimator_
best_models["linear_svm"] = best_linear_svm

In [ ]:
cv_rbf_svm = GridSearchCV(SVC(kernel = "rbf"), param_grid = {
    "C":[1, 10, 100]
})
cv_linear_svm.fit(X,Y)
best_rbf_svm = cv_rbf_svm.best_estimator_
best_models["rbf_svm"] = best_rbf_svm

In [ ]:
cv_poly_svm = GridSearchCV(SVC(kernel = "poly"), param_grid = {
    "C":[1, 10, 100],
    "degree":[2,3,5]
})
cv_poly_svm.fit(X,Y)
best_poly_svm = cv_poly_svm.best_estimator_
best_models["poly_svm"] = best_poly_svm

# PLS Models

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pls_rbf = Pipeline([("pls", PLSRegression()), ("rbf_svm", SVC(kernel = "rbf"))])

In [ ]:
pls_rbf_cv_preds = cross_val_predict(pls_rbf, X, Y, cv = 15)

In [ ]:
pls_linear = Pipeline([("pls", PLSRegression()), ("linear_svm", SVC(kernel = "linear"))])

In [ ]:
pls_linear_cv_preds = cross_val_predict(pls_linear, X, Y, cv = 15)

In [ ]:
pls_polynomial = Pipeline([("pls", PLSRegression()), ("poly_svm", SVC(kernel = "poly"))])

In [ ]:
pls_polynomial_cv_preds = cross_val_predict(pls_polynomial, X, Y, cv = 15)

In [ ]:
pls_lr = Pipeline([("pls", PLSRegression()), ("logistic", LogisticRegression())])

In [ ]:
pls_lr_cv_preds = cross_val_predict(pls_lr, X, Y, cv = 15)

# Fit Stacking Classifier

In [ ]:
model_list = [("This is a TODO my guy!", "Model should go here when it's time (TODO!)")]
final_estimator = LogisticRegression(max_iter = 10000, penalty = "l1", solver = "liblinear")
stacking_estimator = StackingClassifier(estimators = model_list, final_estimator = final_estimator, cv = 15)

In [ ]:
y_pred_ensemble = cross_val_predict(stacking_estimator, X, Y, cv = 15)